# InstructVault RAG Cookbook (Google Colab)

This notebook shows a simple RAG prompt and an agentic RAG prompt using InstructVault.

In [ ]:
!pip -q install instructvault

In [ ]:
# If you're not on Colab, change this path to your local repo folder.
repo_path = "/content/ivault_demo"
print("Using repo_path:", repo_path)

In [ ]:
from pathlib import Path

repo = Path(repo_path)
(repo / "prompts").mkdir(parents=True, exist_ok=True)

rag_prompt = '''spec_version: "1.0"
name: rag_answer
variables:
  required: [question, context]
messages:
  - role: system
    content: "Answer using only the provided context. If missing, say you don't know."
  - role: user
    content: "Question: {{ question }}\n\nContext:\n{{ context }}"
tests:
  - name: includes_context
    vars: { question: "What is InstructVault?", context: "InstructVault is a Git-first prompt registry." }
    assert: { contains_any: ["Context:"] }
'''

agent_prompt = '''spec_version: "1.0"
name: rag_agent
variables:
  required: [question]
messages:
  - role: system
    content: "You are a retrieval-augmented agent. Use tools when needed and cite evidence."
  - role: user
    content: "Question: {{ question }}"
tests:
  - name: includes_question
    vars: { question: "What is prompt versioning?" }
    assert: { contains_any: ["Question:"] }
'''

(repo / "prompts" / "rag_answer.prompt.yml").write_text(rag_prompt, encoding="utf-8")
(repo / "prompts" / "rag_agent.prompt.yml").write_text(agent_prompt, encoding="utf-8")
print("Wrote prompts in", repo / "prompts")

In [ ]:
!ivault validate /content/ivault_demo/prompts

In [ ]:
from instructvault import InstructVault

vault = InstructVault(repo_root=repo_path)
context = "InstructVault keeps prompts in Git and uses CI for evals."
msgs = vault.render("prompts/rag_answer.prompt.yml", vars={"question": "What is InstructVault?", "context": context})
print(msgs)

In [ ]:
!ivault eval /content/ivault_demo/prompts/rag_answer.prompt.yml --report /content/ivault_demo/out/rag_report.json